In [16]:
import os
import yaml
#from dotenv import load_dotenv
from langchain.agents import initialize_agent, Tool
from langchain.agents.agent_types import AgentType
from langchain.llms import OpenAI

#from langchain_openai import OpenAI

from langchain.tools import tool
import requests

#telegram bot
from telegram import Update
from telegram.ext import ApplicationBuilder, ContextTypes, MessageHandler, filters

In [19]:
#set fixed parameters 


### OPENAI API ###
with open('/home/christiane/git/stash/API/openai_api.yml', 'r') as file:
    OPENAI_API_KEY = yaml.safe_load(file)["api"]

### TELEGRAM API ###
with open('/home/christiane/git/stash/API/telegram_api.yml', 'r') as file:
    TELEGRAM_TOKEN = yaml.safe_load(file)["api"]
    
#check api key
#print(OPENAI_API_KEY)
#print(TELEGRAM_TOKEN)


### URL Coingecko ###
#url_coingecko = f"https://api.coingecko.com/api/v3/simple/price?ids={symbol.lower()}&vs_currencies=usd"
#print(url_coingecko)

In [3]:
# Initialize LLM

OpenAI?

#temperature
# 0-0.3 -> focused/predictable
# 0.7-1.0-> creative mode

llm = OpenAI(temperature=0, openai_api_key=OPENAI_API_KEY)


/tmp/ipykernel_7721/763076216.py:9: LangChainDeprecationWarning: The class `OpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import OpenAI``.
  llm = OpenAI(temperature=0, openai_api_key=OPENAI_API_KEY)


In [12]:
# Tool 1: Fetch current Crypto Price

@tool
def get_crypto_price_current(symbol: str) -> str:
    """Returns the current price of a given cryptocurrency symbol (e.g., BTC, ETH) using CoinGecko API"""
     #fix for user input BTC&bitcoin
    COIN_LOOKUP = {
        "btc": "bitcoin",
        "bitcoin": "bitcoin",
        "eth": "ethereum",
        "ethereum": "ethereum",
        "sol": "solana",
        "solana": "solana",
        "ada": "cardano",
        "cardano": "cardano",
        "doge": "dogecoin",
        "dogecoin": "dogecoin",
    }
    
    
    try:
        #for coingecko it has to be full name+lowercase
        coin_lowercase = symbol.strip().lower()   #symbol gets generated from package llm (extracted from input)
        coin_id = COIN_LOOKUP[coin_lowercase]     
        url = f"https://api.coingecko.com/api/v3/simple/price?ids={coin_id}&vs_currencies=eur"
        response = requests.get(url).json()
        price = response[coin_id]["eur"]
        return f"The current price of {coin_id} is {price}€"
    
    except Exception as e:
        return f"Error fetching price: {e}"

    


In [13]:
# Agent Initialization 
tools = [get_crypto_price_current]

agent = initialize_agent(
    tools,
    llm,
    agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    verbose=True
)



In [18]:
# Telegram bot
async def handle_message(update: Update, context: ContextTypes.DEFAULT_TYPE):
    user_message = update.message.text
    response = agent.run(user_message)
    await update.message.reply_text(response)

app = ApplicationBuilder().token(TELEGRAM_TOKEN).build()
app.add_handler(MessageHandler(filters.TEXT & ~filters.COMMAND, handle_message))

if __name__ == "__main__":
    print("Bot is running...")
    app.run_polling()

Bot is running...


RuntimeError: Cannot close a running event loop

In [14]:
# sample usage
if __name__ == "__main__":
    question = "What's the current price of ethereum?"
    print(agent.run(question))

    # Example portfolio value
    portfolio = {
        "bitcoin": 0.2,
        "ethereum": 1.5,
        "solana": 10
    }
    print(agent.run(f"Calculate the value of this portfolio: {portfolio}"))




> Entering new AgentExecutor chain...
 I should use the get_crypto_price_current function to get the current price of ethereum.
Action: get_crypto_price_current
Action Input: "ETH"
Observation: The current price of ethereum is 2445.82€
Thought: I now know the final answer.
Final Answer: The current price of ethereum is 2445.82€.

> Finished chain.
The current price of ethereum is 2445.82€.


> Entering new AgentExecutor chain...
 I should get the current price of each cryptocurrency in the portfolio
Action: get_crypto_price_current
Action Input: symbol
Observation: Error fetching price: 'symbol'
Thought: I should use the correct symbol for each cryptocurrency
Action: get_crypto_price_current
Action Input: BTC
Observation: The current price of bitcoin is 96026€
Thought: I should save this price
Action: get_crypto_price_current
Action Input: ETH
Observation: The current price of ethereum is 2445.82€
Thought: I should save this price
Action: get_crypto_price_current
Action Input: SOL
Ob

In [15]:
#test questions
agent.run("What's the price of BTC?")
#agent.run("Give me the price of bitcoin.")
#agent.run("eth price?")



> Entering new AgentExecutor chain...
 I should use the get_crypto_price_current function to get the current price of BTC.
Action: get_crypto_price_current
Action Input: "BTC"
Observation: The current price of bitcoin is 96070€
Thought: I now know the final answer
Final Answer: The current price of BTC is 96070€.

> Finished chain.


'The current price of BTC is 96070€.'

In [ ]:
@tool
def get_crypto_price_history(symbol: str) -> str:
    url = f"https://api.coingecko.com/api/v3/coins/{symbol.lower()}/market_chart?vs_currency=eur&days=max"
    try:
        response = requests.get(url).json()
        price = response[symbol.lower()]["eur"]

In [ ]:
##telegram connection

#Bot CryptoMOM
#CryptoMOMBot

#t.me/CryptoMOMBot
#token Http API